In [1]:
%reload_ext autoreload
%autoreload 2

import os, math, heapq

import sys
from pathlib import Path

dir_path = Path(os.getcwd()).absolute()
module_path = str(dir_path.parent.parent)
if module_path not in sys.path:
    sys.path.append(module_path)
print(module_path)

import numpy as np
import pandas as pd
import cvxpy as cp

# OWN MODULES
from organsync.models.organsync_network import OrganSync_Network
from organsync.models.organite_network import OrganITE_Network
from data.data_module import UKRegDataModule

import warnings

warnings.filterwarnings("ignore")

/home/bcebere/code/vds/organsync/experiments


In [2]:
# SETUP DATA

batch_size = 256

root_data_dir = Path("../datasets").absolute()

data_dir = root_data_dir / "processed_UKReg"  # your path to UKReg

dm = UKRegDataModule(data_dir, batch_size=batch_size, control=False)
dm.prepare_data()
dm.setup(stage="fit")

In [3]:
import cloudpickle
import torch

with open("models/organite_inference.p", "rb") as f:
    organite_inf = cloudpickle.load(f)

In [4]:
from adjutorium.utils.tester import evaluate_survival_estimator
from adjutorium.plugins.prediction import Predictions

predictions = Predictions(category="risk_estimation")


def generate_surv_data(working_df, use_organ):
    cols = list(dm.x_cols)
    if use_organ:
        cols += list(dm.o_cols)
    X = working_df[cols]
    X = X.drop(columns=["CENS"])

    T = working_df["Y"]
    T = dm.scaler.scale_[-1] * T + dm.scaler.mean_[-1]

    Y = working_df["PCENS"]

    X = X[T > 0]
    Y = Y[T > 0]
    T = T[T > 0]

    return X, T, Y


def generate_interm_surv_data(working_df, use_organ):
    cols = list(dm.x_cols)
    if use_organ:
        cols += list(dm.o_cols)
    X = working_df[cols]
    interm_X = organite_inf.model.representation(torch.from_numpy(np.asarray(X)))
    # X = X.drop(columns = ["CENS"])

    T = working_df["Y"]
    T = dm.scaler.scale_[-1] * T + dm.scaler.mean_[-1]

    Y = working_df["PCENS"]

    X = X[T > 0]
    Y = Y[T > 0]
    T = T[T > 0]

    return X, T, Y


full_df = dm._train_processed

transplant_data = generate_surv_data(full_df[full_df["CENS"] == 0], use_organ=True)
no_transplant_data = generate_surv_data(full_df[full_df["CENS"] == 1], use_organ=False)

interm_transplant_data = generate_interm_surv_data(
    full_df[full_df["CENS"] == 0], use_organ=True
)

full_data = generate_surv_data(full_df, use_organ=True)

In [5]:
from organsync.survival_analysis.xgboost import XGBoostRiskEstimation


def eval_xgboost(X, T, Y):
    print("    data cnt ", len(X))
    surv_analysis = XGBoostRiskEstimation()

    for horizon in [i * 365 for i in range(1, 5)]:

        result = evaluate_survival_estimator(
            surv_analysis,
            X.reset_index(drop=True),
            T.reset_index(drop=True),
            Y.reset_index(drop=True),
            [horizon],
            n_folds=3,
            metrics=["c_index", "brier_score"],
        )
        print(f"   horizon {horizon} -> score {result['str']}")


print("XGBoost eval for transplant data")
eval_xgboost(*transplant_data)


print("XGBoost eval for no transplant data")
eval_xgboost(*no_transplant_data)

XGBoost eval for transplant data
    data cnt  11130
   horizon 365 -> score {'c_index': '0.684 +/- 0.005', 'brier_score': '0.08 +/- 0.004'}
   horizon 730 -> score {'c_index': '0.714 +/- 0.016', 'brier_score': '0.093 +/- 0.002'}
   horizon 1095 -> score {'c_index': '0.701 +/- 0.015', 'brier_score': '0.113 +/- 0.003'}


/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/p

   horizon 1460 -> score {'c_index': '0.7 +/- 0.007', 'brier_score': '0.133 +/- 0.002'}
XGBoost eval for no transplant data
    data cnt  2920


/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/p

/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/p

...

/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time buck

   horizon 1095 -> score {'c_index': '0.847 +/- 0.026', 'brier_score': '0.116 +/- 0.02'}


/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time buck

/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time bucket
  warnings.warn(
/home/bcebere/anaconda3/envs/py38/lib/python3.8/site-packages/xgbse/_debiased_bce.py:286: RuntimeWarning: Warning: Only one class found in a time buck

   horizon 1460 -> score {'c_index': '0.803 +/- 0.026', 'brier_score': '0.144 +/- 0.012'}


In [6]:
from organsync.survival_analysis.cox_ph import CoxPH


def eval_cox_ph(X, T, Y):
    surv_analysis = CoxPH()

    for horizon in [i * 365 for i in range(1, 5)]:
        result = evaluate_survival_estimator(
            surv_analysis,
            X,
            T,
            Y,
            [horizon],
            n_folds=3,
            metrics=["c_index", "brier_score"],
        )
        print(f"   horizon {horizon} -> score {result['str']}")


print("CoxPH eval for transplant data")
eval_cox_ph(*transplant_data)

print("CoxPH eval for no transplant data")
eval_cox_ph(*no_transplant_data)

CoxPH eval for transplant data
   horizon 365 -> score {'c_index': '0.686 +/- 0.003', 'brier_score': '0.071 +/- 0.003'}
   horizon 730 -> score {'c_index': '0.667 +/- 0.011', 'brier_score': '0.1 +/- 0.002'}
   horizon 1095 -> score {'c_index': '0.652 +/- 0.016', 'brier_score': '0.122 +/- 0.002'}
   horizon 1460 -> score {'c_index': '0.643 +/- 0.007', 'brier_score': '0.142 +/- 0.002'}
CoxPH eval for no transplant data
   horizon 365 -> score {'c_index': '0.916 +/- 0.013', 'brier_score': '0.027 +/- 0.007'}
   horizon 730 -> score {'c_index': '0.877 +/- 0.03', 'brier_score': '0.069 +/- 0.008'}
   horizon 1095 -> score {'c_index': '0.793 +/- 0.034', 'brier_score': '0.112 +/- 0.007'}
   horizon 1460 -> score {'c_index': '0.752 +/- 0.025', 'brier_score': '0.149 +/- 0.014'}


In [5]:
from organsync.survival_analysis.xgboost import XGBoostRiskEstimation
import cloudpickle


transplant_surv_analysis = XGBoostRiskEstimation()
transplant_surv_analysis.fit(*transplant_data)

no_transplant_surv_analysis = XGBoostRiskEstimation()
no_transplant_surv_analysis.fit(*no_transplant_data)


with open("models/organite_survival.p", "wb") as f:
    cloudpickle.dump((transplant_surv_analysis, no_transplant_surv_analysis), f)

In [6]:
from organsync.survival_analysis.cox_ph import CoxPH
import cloudpickle


cox_transplant_surv_analysis = CoxPH()
cox_transplant_surv_analysis.fit(*transplant_data)

cox_no_transplant_surv_analysis = CoxPH()
cox_no_transplant_surv_analysis.fit(*no_transplant_data)


with open("models/cox_ph_survival.p", "wb") as f:
    cloudpickle.dump((cox_transplant_surv_analysis, cox_no_transplant_surv_analysis), f)